# Using Selenium + Beautiful Soup

In [5]:
import pandas as pd
import psycopg2 as pg2
import lxml
from selenium import webdriver
from fake_useragent import UserAgent
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from bs4 import BeautifulSoup
import requests
import numpy as np
import re

In [6]:
user_agent = UserAgent()
options = Options()
# options.add_argument('--headless')
options.add_argument('--disable-gpu')
options.add_argument('--disable-extensions')
options.add_argument('--disable-notifications')
options.add_argument('--disable-popup-blocking')
options.add_argument('--start-maximized')
options.add_argument(f'user-agent={user_agent.random}')
options.add_argument('--incognito')

In [7]:
def getMobbiData(soup):
    
    product_link = soup.find_all('a', class_='featured-car-product-link')
    product_link = ["www.mobbi.id"+a['href'] for a in product_link]
    product_link = list(set(product_link))
    
    transmission = soup.find_all('div', {'data-product-transmission': True})
    transmission = [a['data-product-transmission'] for a in transmission ]

    brand = soup.find_all('div', {'data-product-brand': True})
    brand = [a['data-product-brand'] for a in brand ]

    model = soup.find_all('div', {'data-product-category': True})
    model = [a['data-product-category'] for a in model ]

    variant = soup.find_all('div', {'data-product-variant': True})
    variant = [a['data-product-variant'] for a in variant ]

    year = soup.find_all('div', {'data-product-year': True})
    year = [a['data-product-year'] for a in year ]

    price = soup.find_all('div', {'data-product-price': True})
    price = [a['data-product-price'] for a in price ]

    mileage = soup.find_all('div', {'data-product-mileage': True})
    mileage = [a['data-product-mileage'] for a in mileage ]

    location = soup.find_all('div', {'data-product-location': True})
    location = [a['data-product-location'] for a in location ]

    data_lengths = [len(product_link), len(transmission), len(brand), len(variant), len(price), len(year), len(model), len(mileage), len(location)]
    if len(set(data_lengths)) != 1:
        print("Warning: Arrays have different lengths. Filling with NaN.")
        max_length = max(data_lengths)
        for array in [product_link, transmission, brand, variant, price, year, model, mileage, location]:
            while len(array) < max_length:
                array.append(np.nan)
                
    df3 = pd.DataFrame({
    'product_link':product_link,
    'product_brand':brand,
    'product_model':model,
    'product_variant':variant,
    'product_price_(Rp)':price,
    'product_transmission':transmission,
    'product_mileage_(km)':mileage,
    'production_year':year,
    'product_location':location
    })
    
    return df3

In [8]:
df0 = pd.DataFrame()

# set up chromedriver
PATH = "D:\Project\Professional\Astra\Data-Management\In-Class-Training\chromedriver_win32\chromedriver.exe"
driver = webdriver.Chrome(PATH, options=options)

# access url
url = "https://www.mobbi.id/"
driver.get(url)
soup = BeautifulSoup(driver.page_source, 'html.parser')
time.sleep(5)

#close banner
driver.find_element_by_xpath("""//*[@id="btnwClear"]""").click()

for i in range(1,4):
    driver.find_element_by_xpath("""//*[@id="headerNonIbid"]/li/div/div/form/div/div[1]/input[1]""").click()
    # try:
    # driver.find_element_by_xpath("""//*[@id="headerNonIbid"]/li/div/div/form/div/div[1]/input[1]""").click()
    driver.find_element_by_xpath(f"""//*[@id="list-brand-search"]/li[{i}]""").click()

    SCROLL_PAUSE_TIME = 5
    # Get scroll height
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load page
        time.sleep(SCROLL_PAUSE_TIME)

        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
    
    time.sleep(5)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    df = getMobbiData(soup)
    # print(output)
    df0 = pd.concat([df,df0], ignore_index=True)
    df0.reset_index(inplace=True, drop=True)
    # except:
    #     print(f"failed #{i}")
    #     time.sleep(5)
    #     # driver.quit()

time.sleep(5)
# auto quit browser
driver.quit()

In [9]:
"""Identify"""
print(f'The data consist of {df0.shape[0]} rows and {df0.shape[1]} columns.')
print(f'The number of null values contained in this data is {df0.isna().any(axis=1).sum()}.')

The data consist of 569 rows and 9 columns.
The number of null values contained in this data is 0.


In [10]:
df0

,product_link,product_brand,product_model,product_variant,product_price_(Rp),product_transmission,product_mileage_(km),production_year,product_location
0,www.mobbi.id/in/mobil-bekas/honda-jazz-rs-2019...,Honda,CITY,RS (CITY),2.68E+8,MT,4847,2021,Kota Jakarta Utara
1,www.mobbi.id/in/mobil-bekas/honda-hrv-hrv-1-8-...,Honda,BRIO,RS (BRIO),1.75E+8,MT,26077,2021,Kota Pekanbaru
2,www.mobbi.id/in/mobil-bekas/honda-cr-v-turbo-p...,Honda,JAZZ,RS (JAZZ),2.53E+8,AT,78434,2018,Kota Bandung
3,www.mobbi.id/in/mobil-bekas/daihatsu-granmax-p...,Honda,JAZZ,RS NEW (JAZZ),2.44E+8,AT,78380,2017,Kota Bandung
4,www.mobbi.id/in/mobil-bekas/honda-cr-v-turbo-2...,Honda,HR-V,E SPECIAL EDITION (HR-V),3.0E+8,AT,50375,2020,Kota Surabaya
...,...,...,...,...,...,...,...,...,...
564,www.mobbi.id/in/mobil-bekas/toyota-yaris-s-trd...,Toyota,INNOVA,V LUX (INNOVA),3.45E+8,MT,55533,2017,Kota Jakarta Pusat
565,www.mobbi.id/in/mobil-bekas/toyota-all-new-ava...,Toyota,INNOVA,G LUX (INNOVA),3.4E+8,AT,38745,2018,Kota Bekasi
566,www.mobbi.id/in/mobil-bekas/toyota-fortuner-fo...,Toyota,AVANZA,VELOZ (AVANZA),1.7E+8,AT,138697,2018,Kota Bandung
567,www.mobbi.id/in/mobil-bekas/toyota-calya-1-2-g...,Toyota,RUSH,S TRD (RUSH),2.0E+8,AT,163754,2019,Kota Jakarta Timur


In [11]:
df1 = df0.copy()
df1['product_transmission'] = df1['product_transmission'].replace({'MT': 'Manual','AT': 'Automatic'})

def convert_to_price(value):
    return float(value)

def remove_text_inside_parentheses(value):
    return re.sub(r'\s*\([^)]*\)', '', value)

df1['product_price_(Rp)'] = df1['product_price_(Rp)'].astype('float64')
df1['product_mileage_(km)'] = df1['product_mileage_(km)'].astype('float64')
df1['product_variant'] = df1['product_variant'].apply(remove_text_inside_parentheses)

df1


,product_link,product_brand,product_model,product_variant,product_price_(Rp),product_transmission,product_mileage_(km),production_year,product_location
0,www.mobbi.id/in/mobil-bekas/honda-jazz-rs-2019...,Honda,CITY,RS,268000000.0,Manual,4847.0,2021,Kota Jakarta Utara
1,www.mobbi.id/in/mobil-bekas/honda-hrv-hrv-1-8-...,Honda,BRIO,RS,175000000.0,Manual,26077.0,2021,Kota Pekanbaru
2,www.mobbi.id/in/mobil-bekas/honda-cr-v-turbo-p...,Honda,JAZZ,RS,253000000.0,Automatic,78434.0,2018,Kota Bandung
3,www.mobbi.id/in/mobil-bekas/daihatsu-granmax-p...,Honda,JAZZ,RS NEW,244000000.0,Automatic,78380.0,2017,Kota Bandung
4,www.mobbi.id/in/mobil-bekas/honda-cr-v-turbo-2...,Honda,HR-V,E SPECIAL EDITION,300000000.0,Automatic,50375.0,2020,Kota Surabaya
...,...,...,...,...,...,...,...,...,...
564,www.mobbi.id/in/mobil-bekas/toyota-yaris-s-trd...,Toyota,INNOVA,V LUX,345000000.0,Manual,55533.0,2017,Kota Jakarta Pusat
565,www.mobbi.id/in/mobil-bekas/toyota-all-new-ava...,Toyota,INNOVA,G LUX,340000000.0,Automatic,38745.0,2018,Kota Bekasi
566,www.mobbi.id/in/mobil-bekas/toyota-fortuner-fo...,Toyota,AVANZA,VELOZ,170000000.0,Automatic,138697.0,2018,Kota Bandung
567,www.mobbi.id/in/mobil-bekas/toyota-calya-1-2-g...,Toyota,RUSH,S TRD,200000000.0,Automatic,163754.0,2019,Kota Jakarta Timur


# Save the DataFrame to a CSV file

In [12]:
file_path = './Exported/Internship_Sean_mobbi_data.csv'
df1.to_csv(file_path, index=False)

print("DataFrame saved as CSV.")

DataFrame saved as CSV.


In [13]:
df1.dtypes

product_link             object
product_brand            object
product_model            object
product_variant          object
product_price_(Rp)      float64
product_transmission     object
product_mileage_(km)    float64
production_year          object
product_location         object
dtype: object

# Create a table from a database from the DataFrame

In [14]:
username = 'postgres'
password = 'password'
host = 'localhost'
database_name = 'sql-challenge'
port = '5432'

In [15]:
conn = pg2.connect(
    host=host,
    port=port,
    database=database_name,
    user=username,
    password=password
)
conn.autocommit = True
cur = conn.cursor()

In [18]:
"""Create a new schema and table"""
new_schema = 'web_scraping_Sean'
new_table = 'Internship_Sean_mobbi_scraping'
csv_file_path = 'D:\Project\Professional\Astra\Data-Management\Challenge\sql_challenge_main\Github\sql-challenge\Sean\Exported\Internship_Sean_mobbi_data.csv'
cur.execute(f'CREATE SCHEMA IF NOT EXISTS {new_schema} AUTHORIZATION postgres;')
cur.execute(f'''
    CREATE TABLE IF NOT EXISTS {new_schema}.{new_table} (
        product_link TEXT,
        product_brand TEXT,
        product_model TEXT,
        product_variant TEXT,
        product_price_rp NUMERIC,
        product_transmission TEXT,
        product_mileage_km NUMERIC,
        production_year TEXT,
        product_location TEXT
    )
''')
cur.execute(f'''
    COPY {new_schema}.{new_table} 
    FROM '{csv_file_path}' 
    DELIMITER ',' 
    CSV HEADER;
'''
)


In [21]:
cur.close()
conn.close()